In [ ]:
import torch
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal

# Custom imports
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, os.path.dirname(currentdir))
from simulators import get_simulator
from ml.real_nvps import RealNVP, RealNVPs
from ml.umnn_maf_flow import UmnnMafFlow
from ml.ml_helper import fit_conditional_normalizing_flow
from metrics.metrics import compute_roc_auc
from inference.neb import ElboEstimator, IwEstimator
from utils.noise_distribution import GaussianDistribution

In [ ]:
device = 'cuda'

# Define your simulator

In [ ]:
simulator_name = 'SLCP' # Choose a simulator in ['SLCP', 'IK', '2dMoon']
simulator = get_simulator(simulator_name)

# Train a surrogate model of the simulator 
#### (needed when no likelihood function is known in closed-form)

In [ ]:
# Generate the training dataset
dataset_size = 15000
proposal_distribution = MultivariateNormal(torch.zeros(dataset_size, simulator.xdim), torch.eye(simulator.xdim))
x_train = proposal_distribution.sample()
y_train = simulator.corrupt(x_train)

# Train the surrogate model
flow_length = 4
hidden_layer_dim = 50

surrogate = RealNVPs(flow_length, simulator.ydim, simulator.xdim, hidden_layer_dim).to(device)
optimizer = optim.Adam(surrogate.parameters(), weight_decay=5*10**(-5), lr=1e-4)

training_loss, validation_loss = fit_conditional_normalizing_flow(surrogate, 
                                                                  optimizer, 
                                                                  y_train.to(device), 
                                                                  x_train.to(device), 
                                                                  validation_size=0.1, 
                                                                  early_stopping_patience=10,
                                                                  batch_size=128, 
                                                                  nb_epochs=300)
surrogate.eval();

#### Assess the quality of the surrogate model (the closer to 0.5, the better)

In [ ]:
source_data = proposal_distribution.sample()
true_y = simulator.corrupt(source_data)

noise = GaussianDistribution.sample(dataset_size, dim=simulator.ydim)
regenerated_y = surrogate.invert(noise.to(device), context=source_data.to(device))

print('ROC AUC:', compute_roc_auc(true_y.data.numpy(), regenerated_y.data.cpu().numpy()))

# Source Distribution Estimation 

In [ ]:
# Unseen source distribution
source_distribution = simulator.sample_prior(10000)
# Observations
observations = simulator.corrupt(source_distribution)

#### Definition of the recognition network & source model

In [ ]:
"""
These are heavy models, lighter models are recommended when inference 
is done in higher dimensions or on more data.
"""

recognition_network = UmnnMafFlow(nb_flow=6, 
                                  nb_in=simulator.xdim, 
                                  cond_in=simulator.ydim,
                                  hidden_derivative=[75, 75, 75], 
                                  hidden_embedding=[75, 75, 75],
                                  embedding_s=10, 
                                  nb_steps=20,
                                  device=device)

optimizer_recognition_network = optim.Adam(recognition_network.parameters(), weight_decay=0.0, lr=1e-4)

source_model = UmnnMafFlow(nb_flow=6, 
                           nb_in=simulator.xdim, 
                           cond_in=0,
                           hidden_derivative=[75, 75, 75], 
                           hidden_embedding=[75, 75, 75],
                           embedding_s=10, 
                           nb_steps=20,
                           device=device)

optimizer_source_model = optim.Adam(source_model.parameters(), weight_decay=0.0, lr=1e-4)

#### Inference

In [ ]:
log_likelihood_fct = lambda y, x : surrogate.compute_ll(y, context=x)[0]

estimator = IwEstimator() # Choose an estimator in ['ElboEstimator', 'IwEstimator']

nb_epochs = 300
nb_mc_integration_steps = 128
batch_size = 128
early_stopping=True,

training_loss, validation_loss = estimator.infer(observations.to(device), 
                                                 source_model, 
                                                 optimizer_source_model,
                                                 recognition_network,
                                                 optimizer_recognition_network,
                                                 log_likelihood_fct, 
                                                 simulator.xdim,
                                                 nb_epochs=nb_epochs, 
                                                 batch_size=batch_size, 
                                                 early_stopping=early_stopping)

#### Assess the quality of the learned source distribution (the closer to 0.5, the better)

In [ ]:
noise = GaussianDistribution.sample(10000, dim=simulator.xdim)

# Assess the quality of the source model
regenerated_source_data = source_model.invert(noise.to(device)).data.cpu() # The forward pass is used during
                                                                           # training for density evaluation.
                                                                           # Sampling is done by inverting the
                                                                           # flow.
regenerated_observations = simulator.corrupt(regenerated_source_data).data

print('ROC AUC x-space:', compute_roc_auc(regenerated_source_data.numpy(), source_distribution.data.numpy()))
print('ROC AUC y-space:', compute_roc_auc(regenerated_observations.numpy(), observations.data.numpy()))